## Importing all dependencies 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

### Loading of Data 

In [2]:
%%time 
df = pd.read_csv('IMDB-Dataset.csv',encoding='latin-1')

CPU times: total: 6.48 s
Wall time: 9.6 s


### Data Cleaning and Preprocessing 

In [3]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)
print(stop_words)

{'until', 'itself', 'any', 'you', 'mightn', 'won', 'now', 'i', 'might', "you'd", 'ourselves', 'would', 'again', 'themselves', "aren't", "isn't", 'y', 'haven', "she's", 'wasn', 'am', 'further', 'be', 'some', 'of', 'own', 'these', "mightn't", 'weren', 'our', 'in', "hasn't", 'on', 'once', 'into', 't', 'aren', 'through', "don't", 'hasn', 'we', 'can', 'has', 'for', 'but', 'under', "you'll", 'should', 'was', 'a', 'at', 'herself', 'having', 'very', 'before', 'o', 'other', 'the', 'down', "needn't", 'over', 'where', 'both', 'to', 'during', 'no', 'while', 'shan', 'will', 'its', 'which', 'those', 'why', 'all', 'most', 'and', "didn't", 'with', 'could', "haven't", 'same', 'shouldn', 'm', 've', 'isn', 'from', 'few', 'hers', 'them', 'don', 'his', "wouldn't", 'd', 'myself', 'it', 'theirs', "doesn't", 'up', "it's", 'what', 'how', 'if', 'between', 'her', 'yourselves', 'being', 'than', 'doing', "should've", 'did', 'didn', 'me', 'about', "couldn't", "that'll", 'there', 'couldn', "won't", 'wouldn', 'shall'

In [4]:
'''-----------------------------Data Cleaning and Preprocessing pipeline----------------------------------'''

#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    '''content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)'''
    return content

#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [5]:
%%time
pd.options.display.max_colwidth = 1000
#Data cleaning
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: total: 54.6 s
Wall time: 55.4 s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

### Feature Engineering 

In [6]:
#Mapping rating data to Binary label 1 (+ve) if rating >=7 and 0 (-ve) if rating <=4 and 2 (neutral) if rating = 5 or 6
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
#Removing 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())

0    60000
1    60000
Name: Label, dtype: int64


In [7]:
pip install prettytable

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [8]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

### Lemmatization 

In [9]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

### Vectorization with CountVector and TDIDF Vectorizer with Unigram 

In [10]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shalini\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shalini\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
import nltk
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Shalini\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Importance with Logistic Regression and Count Vectorizer with unigram 

In [14]:
lgr = LogisticRegression()
lgr.fit(x_train_count, y_train)
lgr.score(x_test_count, y_test)
lgr.coef_[0]

i = 0
important_features = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.vocabulary_.keys(), lgr.coef_[0]):
    if i <= 200:
        important_features.add_row([feature, importance])
        i += 1
print(important_features)


+---------------+------------------------+
|    Feature    |         Score          |
+---------------+------------------------+
|      film     |  0.10051131999868529   |
|      far      |  0.04759115774582529   |
|     better    |  -0.15708820691529887  |
|     rating    |  -0.16709430572416006  |
|      cast     |   0.2678712994162001   |
|      work     |  -0.1290918865060221   |
|   storyline   |  -0.14910268530521278  |
|     quite     |  -0.05267217383008929  |
|      done     |  0.02593914864734874   |
|      time     |  0.05990843645000427   |
|    however    |  0.10016342682215937   |
|     never     |  -0.07112951962244364  |
|      well     | -0.014782885405327064  |
|     alien     |   0.1242121024964387   |
|     begin     |  -0.20326876620162562  |
|     found     |  0.09806637897537157   |
|     highly    |   0.1984167959418703   |
|  entertaining |   0.2015865604206019   |
|   production  |   0.8423592416449139   |
|    quality    |  0.054676849270351234  |
|    audien

### Feature Importance with TFIDF vectoizer and Logistic Regression wit Unigram

In [15]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.vocabulary_
, lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+--------------+-----------------------+
|   Feature    |         Score         |
+--------------+-----------------------+
|     film     |   0.3886174922809152  |
|     far      |   0.4704499846587188  |
|    better    |   -1.047469656297766  |
|    rating    |  -1.6661154789334227  |
|     cast     |   2.649728671096299   |
|     work     |  -1.3611197026238104  |
|  storyline   |   -1.014064246325717  |
|    quite     |  -0.4774034984689914  |
|     done     |  0.13023166603083625  |
|     time     |   0.6912580864854935  |
|   however    |   0.7656404935012993  |
|    never     |  -0.3350689740634511  |
|     well     |  -0.2885935736426229  |
|    alien     |   0.7506666283673725  |
|    begin     |   -1.144459237912402  |
|    found     |   1.2244973113766267  |
|    highly    |   1.2308636395882817  |
| entertaining |   1.709886787628334   |
|  production  |   4.790587266969413   |
|   quality    |  0.46329698483248705  |
|   audience   |   -3.793759110083493  |
|     find     |

### Vectorization with Count Vectorizer and TDIDF Vectorizer with Bigram 

In [16]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Importance with Logistic Regression and Count Vectorizer with Bigram 

In [17]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|       acting bad      |  -1.3592826415039752  |
|      acting good      |   0.6547550628811787  |
|       acting not      |  -0.4678206199502798  |
|      action film      |   0.3699274125474267  |
|      action movie     |   0.2910595529206006  |
|     action packed     |   1.3386783052976188  |
|      action scene     |   0.1751897366969938  |
|    action sequence    |  0.10151696252212736  |
|     actor actress     |  -0.1255733889277545  |
|       actor not       |  -0.3918608062393022  |
|      almost every     |  -0.13260075331333696 |
|       along way       |  0.49088604257087676  |
|        also not       |  -0.13974956968421473 |
|      although not     |  0.49910230925061255  |
|     anything else     |  -0.3232091756467245  |
|        b movie        |  -0.09071784029437928 |
|       bad acting      |  -1.0605943529061646  |


### Feature Importance with Logistic Regression and TFIDF Vectorizer with Bigram 

In [18]:
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=50:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+----------------------+
|        Feature        |        Score         |
+-----------------------+----------------------+
|       acting bad      |  -2.98717605343219   |
|      acting good      |  1.4431054145457136  |
|       acting not      |  -1.274469874526418  |
|      action film      |  1.2237150874358569  |
|      action movie     |  0.8443556589356525  |
|     action packed     |  3.0238449287615965  |
|      action scene     |  0.470043582118703   |
|    action sequence    |  0.2964462639054992  |
|     actor actress     | -0.26151819203681653 |
|       actor not       | -0.9344034381742992  |
|      almost every     | -0.29476710752147645 |
|       along way       |  1.0753056686648317  |
|        also not       |  -0.290929759330877  |
|      although not     |  1.104396976989657   |
|     anything else     | -0.7966422564307294  |
|        b movie        | -0.4228615975141658  |
|       bad acting      | -2.7994729658080213  |
|        bad bad    

In [19]:
pd.options.display.max_colwidth = 1000
df[["Reviews","Ratings","Movies"]][(df['Ratings']>=9)&(df['Reviews_clean'].str.contains("bad review"))].head(100)

,Reviews,Ratings,Movies
120047,"While I wouldn't call this the greatest movie ever made, it's not anywhere near as bad as other reviewers have made it out to be. An average rating of 5 or 6 stars would be fair, but 1.5 is harsh and totally undeserved.Ring of Terror feels like an episode of The Twilight Zone stretched to an hour. In fact, it's so much like a TV show that one wonders if it might not have been originally created as a pilot.If you're a fan of 1950s horror/suspense series like Thriller, The Veil, One Step Beyond, Tales of Tomorrow, and Alfred Hitchcock Presents, you'll likely find this a pleasant way to spend an hour, as I did.Normally I would only give this film 6 out of 10 stars, but because others have been panning it so unmercifully, I'm giving it a 9.",9.0,Ring of Terror
120211,"This movie was a blast for my little guys, they loved every minute of it, I have read all of the bad reviews, and could not disagree more. This movie, is pure and good. There is just enough action to keep the kids interested, and not so much that you leave the theater with them bouncing off the walls either. It is funny with jokes that everyone can appreciate. I think people have gotten used to so much violence and adult content in our kids movies that they are disappointed when it is missing, like the movie wasn't entertaining enough for the parents. Well, NEWS FLASH.....It's a kids movie, and a perfect one at that. Kids need these kinds of movies, not Spongebob and the like which are more to entertain the parents.",9.0,Doogal
120238,"I am a huge horror buff and prefer pieces that delve into the characters psychological issues. This film was awesome on so many levels, the acting and writing were fantastic and creepy and I was afraid or and empathetic with the murderer the whole time. What an interesting study on the line between sick and a danger to others, and the line between being a mean girl and being psychotic. Set in a great location, a house full of creepy art, in the winter in Conneticut and with amazing performances from many of my favorite actresses. It actually shocks me that others have given this such a bad review, I loved this movie, I guess it goes to show you everyone will have a different opinion but I say don't miss this film!",9.0,#Horror
120239,"No idea why there are so many bad reviews here? I loved it; I thought it was a very advanced thoughtful film. The graphic were #killer. The comparison of video game culture and young girl culture was spot on. This film makes connections that I've never seen on the big screen but, do see in every day life.The casting was spot on, Hello 12 year-old girls are supposed to be a little annoying. I do wish that more directors would take color into more consideration the way this film does. T The highly stylized sets make the murder scenes more believable because everything is so unbelievable. How can you live in 2016 and not ""get""a film about social media and accelerationism. #duh Someone explain this to me.",9.0,#Horror
120273,"What do you get when you cross Love Story with Star Wars with Blade Runner with Back to the Future with MTV? Love Story 2050, that's what. What a fun movie for the entire family. This fantasy of epic proportions is much, much better than AI, a similar sci-fi classic. The thrills are non-stop in this blockbuster, from its lead off car chase to bike racing stunts to the vantage point of a moving roller coaster to speeding hover-craft--you will be on the edge of your seat from beginning to end. The version I saw was only partially in English and I still was glued to the screen. I can't wait to see a version with subtitles. The mega budget special effects are out of this world and highly convincing. The future vision of XBox was hilarious. Those who are complaining about how long this movie is simply don't understand Bollywood. The three hours went by quickly; it seemed to be only an hour. There could have been a better twist with the Darth Vader character. 

### Vectorization with Count Vectorizer and TFIDF Vectorizer with Trigram
 

In [20]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

###  Feature Importance with Logistic Regression and Count Vectorizer with Trigram

In [21]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+-----------------------+
|          Feature          |         Score         |
+---------------------------+-----------------------+
|       acting not bad      |  -0.4160546895868406  |
|      acting not good      |  -1.0608422834527877  |
|      acting not great     |  -0.4673938774319415  |
|     acting pretty good    |   0.9604426395517075  |
|      acting top notch     |   1.8064792512163783  |
|      action movie not     |   1.0284071921648876  |
|      action set piece     |   1.2166459597906552  |
|     action take place     |  0.04247654797414374  |
|       actor good job      |   0.5960771625512327  |
|    actually pretty good   |  0.30683132542660213  |
|    actually quite good    |  -0.16411628927176514 |
|     almost every scene    |  -0.3190690932755595  |
|        b movie not        |  -0.04725855664810745 |
|       bad acting bad      |  -2.3282408027634585  |
|        bad bad bad        |   -1.768613567319371  |
|        bad guy not        

### Feature Importance with Logistic Regression and TFIDF Vectorizer with Trigram 

In [22]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+------------------------+
|          Feature          |         Score          |
+---------------------------+------------------------+
|       acting not bad      |   -0.573281686424128   |
|      acting not good      |  -1.3107700908904203   |
|      acting not great     |  -0.6602718882804156   |
|     acting pretty good    |   1.0932579289655375   |
|      acting top notch     |    2.07464926123333    |
|      action movie not     |   1.204591359694098    |
|      action set piece     |   1.243002917792281    |
|     action take place     |  0.09013657404852578   |
|       actor good job      |   0.6812433617400977   |
|    actually pretty good   |  0.33819941787619723   |
|    actually quite good    |  -0.14193988072213728  |
|     almost every scene    |  -0.3818008160012452   |
|        b movie not        |   -0.092724091004381   |
|       bad acting bad      |   -2.858563396274231   |
|        bad bad bad        |  -2.5936064580505076   |
|        b

### Vectorization with Count Vectorizer and TDIDF Vectorizer with 4-gram 

In [23]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

###  Feature Importance with Logistic Regression and Count Vectorizer with 4-gram

In [24]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+------------------------------------------------+-----------------------+
|                    Feature                     |         Score         |
+------------------------------------------------+-----------------------+
|         according dvd sleeve synopsis          |  -2.2545695463340056  |
|           acting also great special            |  0.35749873395252935  |
|             acting bad script bad              |  -1.0274211884407172  |
|     addictive consistently compelling show     |  0.07279038856770845  |
|                 ai not gon na                  |  -0.4873083280290686  |
|           also great special effect            |   0.7705316686992361  |
|       annoying illogical decision making       | -0.036401551135883614 |
|           another film seen recently           |  -0.03886544318374477 |
|         appreciation horror low budget         |  -0.10976598356701495 |
|        atmosphere dreary due excessive         | -0.040640459453615996 |
|         atmospheric bea

### Feature Importance with Logistic Regression and TDIDF Vectorizer with 4-gram 

In [25]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+------------------------------------------------+-----------------------+
|                    Feature                     |         Score         |
+------------------------------------------------+-----------------------+
|         according dvd sleeve synopsis          |   -2.272823486373116  |
|           acting also great special            |   0.6744187057552399  |
|             acting bad script bad              |  -1.3135142386543432  |
|     addictive consistently compelling show     |   0.247005128472225   |
|                 ai not gon na                  |  -0.47533923703858144 |
|           also great special effect            |   1.0330901038073614  |
|       annoying illogical decision making       |  -0.22585405492656638 |
|           another film seen recently           |  -0.2455966341610594  |
|         appreciation horror low budget         |  -0.27657853294948975 |
|        atmosphere dreary due excessive         |  -0.2585822875329113  |
|         atmospheric bea

###  Vectorization with Count Vectorizer and TDIDF Vectorizer with unigram, bigram and trigram

In [ ]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=5000)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

###  Feature Selection with Chi squared

In [ ]:
rom sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1